In [1]:
import numpy as np
import pandas as pd
from functions import createDataMain,createTestDataMain,pd2np
import cPickle

In [2]:
batch_size = 100
discreteList = ['dayOfWeek','payDay','month','earthquake',\
                'type','locale','locale_name','transferred','onpromotion']


In [3]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

In [5]:
types = {'id': 'int32',
     'item_nbr': 'int32',
     'store_nbr': 'int8',
     'unit_sales': 'float32',
     'onpromotion': bool}
train = pd.read_csv('train.csv',usecols=['date','item_nbr','store_nbr','unit_sales','onpromotion'],\
                    parse_dates=['date'],dtype=types, infer_datetime_format=True)
train = train.fillna(2,axis=1)
train.onpromotion = train.onpromotion.astype(np.int8)
train.loc[train.unit_sales<0,'unit_sales'] = .0 # clip negative sales to zero
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
with open(r"item_dict_train.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

In [7]:
train = train[train.date < '2017-07-31']
item_uniq = pd.unique(train.item_nbr)
item_dict = {item:i+1 for i,item in enumerate(item_uniq)}

In [8]:
with open(r"item_dict_final.pickle", "wb") as output_file:
    cPickle.dump(item_dict, output_file)

In [4]:
train,val = createDataMain(True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(val,batch_size,val.countDays.max(),dateVar_list,'date',discreteList)

In [6]:
prefix = 'val'

In [7]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [5]:
y_np, weight_np,Con_np,Dis_np,X_np,Count_np = \
            pd2np(train,batch_size,train.countDays.max(),dateVar_list,'date',discreteList)

In [4]:
prefix = 'train'

In [7]:
np.savetxt(prefix+'_Y',y_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Weight',weight_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_Con',Con_np,fmt="%f",delimiter=",") 
np.savetxt(prefix+'_X',X_np,fmt="%d",delimiter=",") 
np.savetxt(prefix+'_Count',Count_np,fmt="%d",delimiter=",") 
for j in range(len(discreteList)):
    np.savetxt(prefix+'_Dis'+str(j),Dis_np[:,:,j],fmt="%d",delimiter=",")  

In [5]:
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [7]:
y_np.shape

(174140, 1672)

In [8]:
len(Dis_np)

9

In [12]:
def Save2Disk(data,batchSize,seqSize,df,key,discreteList,prefix):
    n = data.shape[0]
    d = len(discreteList)
    Y,Con = file(prefix+'_Y.csv','a'),file(prefix+'_Con.csv','a')
    Dis = [file(prefix+'_Dis'+str(i)+'.csv','a') for i in range(d)]
    for y,con,dis in RNN_generator2(data,batchSize,seqSize,df,key,discreteList):
        np.savetxt(Y,y,fmt="%f",delimiter=",")  
        np.savetxt(Con,con,fmt="%f",delimiter=",")  
        for j,Dis_j in enumerate(Dis):
            np.savetxt(Dis_j,dis[:,:,j],fmt="%d",delimiter=",")  
    
    Y.close()    
    Con.close()
    for Dis_j in Dis:
        Dis_j.close()
    weight = np.ones((n,1),dtype=np.float32)
    weight[data['perishable']==1] = 1.25
    np.savetxt(prefix+'_Weight.csv', weight, delimiter=',')
    np.savetxt(prefix+'_Count.csv', fmt="%d",data['countDays'].values.astype(np.int32), delimiter=',')


def RNN_generator2(data,batchSize,seqSize,df,key,discreteList):
    n = data.shape[0]
    for from_ in range(0,n,batchSize):
        for y,Xdiscrete,Xcontinue in timeGenerator2(data.loc[from_:from_+batchSize-1,'date':'amax'],\
                                                      seqSize,df,key,discreteList):
            yield y,Xcontinue, Xdiscrete
            
def timeGenerator2(data,seqSize,df,key,discreteList):
    # df is time related variables like holiday and seasonality
    # returns y, X_discrete, X_continue
    n = data.shape[0]
    sparse = pd.concat([data.apply(lambda x: pd.Series(x.date,name='date'),axis=1)\
                                    .stack().reset_index().drop(['level_1'],1),\
                        data.apply(lambda x: pd.Series(x.unit_sales),axis=1)\
                                    .stack().reset_index().drop(['level_0','level_1'],1),\
                        data.apply(lambda x: pd.Series(x.onpromotion),axis=1)\
                                    .stack().reset_index().drop(['level_0','level_1'],1)],1)
    sparse.columns = ['level','date','sales','onpromotion']

    dense = data.apply(lambda x:pd.Series(pd.date_range(x.amin,periods=seqSize)),axis=1)\
                        .stack().reset_index().drop(['level_1'],1)
    dense.columns = ['level','date']
    dense = pd.merge(pd.merge(dense, df[np.random.randint(10)], how='left', on=key),
                    sparse,how='left',on=['level','date']).fillna(0)
    dense_continue = dense['dcoilwtico'].values.astype(np.float32).reshape((n,seqSize))
    dense_discrete = dense[discreteList].values.astype(np.int32)\
                        .reshape((n,seqSize,len(discreteList)))
    y = dense['sales'].values.astype(np.float32).reshape((n,seqSize))

    yield y, dense_discrete, dense_continue
